In [14]:
from random import random
from math import ceil
from functools import reduce
from collections import namedtuple, deque
from queue import PriorityQueue

import numpy as np
from tqdm.auto import tqdm

In [24]:
State = namedtuple('State', ['taken', 'not_taken'])


def covered(state):
    return reduce(
        np.logical_or,
        [SETS[i] for i in state.taken],
        np.array([False for _ in range(PROBLEM_SIZE)]),
    )


def goal_check(state):
    return np.all(covered(state))

In [25]:
PROBLEM_SIZE = 20
NUM_SETS = 40
SETS = tuple(np.array([random() < 0.2 for _ in range(PROBLEM_SIZE)]) for _ in range(NUM_SETS))
assert goal_check(State(set(range(NUM_SETS)), set())), "Probelm not solvable"

In [26]:
SETS

(array([False, False, False, False, False, False,  True, False, False,
        False, False, False, False, False, False, False,  True, False,
         True, False]),
 array([False, False, False, False, False, False, False, False, False,
        False,  True, False, False,  True,  True, False, False, False,
         True,  True]),
 array([False,  True, False, False,  True, False, False, False, False,
        False, False,  True, False,  True,  True, False, False,  True,
         True, False]),
 array([ True, False, False, False,  True, False, False, False, False,
        False,  True, False, False, False,  True, False,  True, False,
        False, False]),
 array([False, False, False, False, False, False, False, False, False,
         True, False, False, False,  True, False,  True,  True, False,
        False, False]),
 array([False, False, False, False, False, False, False, False, False,
        False,  True, False, False, False, False, False, False, False,
        False, False]),
 arr

In [33]:
def h(state):
    largest_set_size = max(sum(s) for s in SETS)
    missing_size = PROBLEM_SIZE - sum(covered(state))
    optimistic_estimate = ceil(missing_size / largest_set_size)
    return optimistic_estimate


def h2(state):
    already_covered = covered(state)
    if np.all(already_covered):
        return 0
    largest_set_size = max(sum(np.logical_and(s, np.logical_not(already_covered))) for s in SETS)
    missing_size = PROBLEM_SIZE - sum(already_covered)
    optimistic_estimate = ceil(missing_size / largest_set_size)
    return optimistic_estimate


def h3(state):
    already_covered = covered(state)
    if np.all(already_covered):
        return 0
    missing_size = PROBLEM_SIZE - sum(already_covered)
    candidates = sorted((sum(np.logical_and(s, np.logical_not(already_covered))) for s in SETS), reverse=True)
    taken = 1
    while sum(candidates[:taken]) < missing_size:
        taken += 1
    return taken

def h_greedy(state):
    already_covered = covered(state)
    
    if np.all(already_covered):
        return 0  # Se tutti gli elementi sono già coperti, il costo rimanente è 0.

    remaining_elements = PROBLEM_SIZE - np.sum(already_covered)
    
    # Inizializza il costo rimanente a zero.
    taken = 0
    
    while np.any(already_covered == False) and taken < remaining_elements:
        # Trova il set non ancora preso con la massima copertura rimasta.
        max_covering_set = None
        max_covering_size = -1
        for i, s in enumerate(SETS):
            if i not in state.taken:
                covering_size = np.sum(np.logical_and(s, np.logical_not(already_covered)))
                if covering_size > max_covering_size:
                    max_covering_size = covering_size
                    max_covering_set = s
        
        # Aggiungi il set con la massima copertura rimasta.
        if max_covering_set is not None:
            state.taken.add(i)
            already_covered = np.logical_or(already_covered, max_covering_set)
            taken += 1
        else:
            break  # Non ci sono set rimasti da prendere.
    
    return taken



def f(state):
    return len(state.taken) + h_greedy(state)



In [34]:
frontier = PriorityQueue()
state = State(set(), set(range(NUM_SETS)))
frontier.put((f(state), state))

counter = 0
_, current_state = frontier.get()
with tqdm(total=None) as pbar:
    while not goal_check(current_state):
        counter += 1
        for action in current_state[1]:
            new_state = State(
                current_state.taken ^ {action},
                current_state.not_taken ^ {action},
            )
            frontier.put((f(new_state), new_state))
           
        _, current_state = frontier.get()
        print(current_state)
        pbar.update(1)

print(f"Solved in {counter:,} steps ({len(current_state.taken)} tiles)")

45it [00:00, 154.56it/s]

State(taken={39}, not_taken={0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38})
State(taken={27, 39}, not_taken={0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39})
State(taken={27, 39}, not_taken={0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38})
State(taken={28, 39}, not_taken={0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39})
State(taken={30, 39}, not_taken={0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 31, 32, 33, 34, 35, 36, 37, 38, 39})
State(taken={30, 39}, not_taken={0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 2

Solved in 45 steps (5 tiles)


In [35]:
current_state

State(taken={38, 39, 23, 28, 30}, not_taken={0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 24, 25, 26, 27, 29, 31, 32, 33, 34, 35, 36, 37})